In [4]:
import math
import numpy as np
import pyquil.quil as pq
from pyquil.gates import *
from pyquil.api import QVMConnection
from functools import reduce


def single_shot_grovers(data):
  
  # start the quantum program
	qvm = QVMConnection()
	p = pq.Program()

  # Grover's requires an N-dimensional state space H, where N is the number
  # of entries in the database. We can construct this with log_2(N) qubits.

  # Let data_size represent the number of qubits required. Use base change rule!
  # https://www.purplemath.com/modules/logrules5.htm
	data_size = int(math.log10(len(data)) / math.log10(2))
  
  # iterate over the qubits and build the state space.
  # allocate log_2(N) single-qubit Hadamard gates
	for index in range(0,data_size):
		p.inst(H(index))

  # Here we define the oracle: a gate that represents the predicate you want to 
  # search for a satisfying solution to. This gate should return true for 
  # the correct solution and false for all others. 
	oracle = np.identity(len(data)) - 2 * np.diag(data)
  
  # Make the oracle a static gate and add it for every qubit.
	p.defgate("oracle", oracle)
	p.inst(tuple(["oracle"] + [i for i in range(0,data_size)]))

  

    ## complete the following stub code (this is where you complete the code)
    ## This is the Grover diffusion operator.
	grover_op = 2 * (1./len(data)) * np.ones(tuple([len(data)] + [1])) * np.ones(len(data)) - np.identity(len(data))
  
  # TODO: define the grover gate, allocate it for every qubit
	p.defgate("oracle", oracle)
	p.inst(tuple(["oracle"] + [i for i in range(0,data_size)]))
  
  ## End of TODO


	for index in range(0,data_size):
		p.measure(index, index)

	print(p)
	wavefunction = qvm.wavefunction(p)
	print(qvm.wavefunction(p))
  
  # convert to classical index
	classical_regs = [i for i in range(0, data_size)]
	for i in classical_regs:
		p.measure(i,i)
	result = qvm.run(p, classical_regs)[0]
	result = reduce(lambda x,y: 2*x+y, result)
	return(result)

single_shot_grovers([0,0,1,0])

DEFGATE oracle:
    1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, -1.0, 0
    0, 0, 0, 1.0

DEFGATE oracle:
    1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, -1.0, 0
    0, 0, 0, 1.0

DECLARE ro BIT[2]
H 0
H 1
oracle 0 1
oracle 0 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]

(1+0j)|11>


/Users/neil/anaconda3/lib/python3.7/site-packages/pyquil/quil.py:166: UserWarning: Gate oracle has already been defined in this program
  .format(instruction.name))
/Users/neil/anaconda3/lib/python3.7/site-packages/pyquil/gates.py:487: UserWarning: Indexing measurement addresses by integers is deprecated. Replacing this with the MemoryReference ro[i] instead.
  warn("Indexing measurement addresses by integers is deprecated. "
/Users/neil/anaconda3/lib/python3.7/site-packages/pyquil/quil.py:869: UserWarning: Please DECLARE all memory. I'm adding a declaration for the `ro` register, but I won't do this for you in the future.
  warnings.warn("Please DECLARE all memory. I'm adding a declaration for the `ro` register, "


3